In [1]:
import ipywidgets as widgets
from IPython.display import display, clear_output, Image
import time
import requests
from bs4 import BeautifulSoup
import json
from jupyter_ui_poll import ui_events
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
#Uploading data to Google Form
def send_to_google_form(data_dict, form_url):
    form_id = form_url[34:90]
    view_form_url = f'https://docs.google.com/forms/d/e/{form_id}/viewform'
    post_form_url = f'https://docs.google.com/forms/d/e/{form_id}/formResponse'

    page = requests.get(view_form_url)
    content = BeautifulSoup(page.content, "html.parser").find('script', type='text/javascript')
    content = content.text[27:-1]
    result = json.loads(content)[1][1]
    form_dict = {}
    
    loaded_all = True
    for item in result:
        if item[1] not in data_dict:
            print(f"Form item {item[1]} not found. Data not uploaded.")
            loaded_all = False
            return False
        form_dict[f'entry.{item[4][0][0]}'] = data_dict[item[1]]
    
    post_result = requests.post(post_form_url, data=form_dict)
    return post_result.ok


event_info = {
    'type': '',
    'description': '',
    'time': -1
}

def wait_for_event(timeout=-1, interval=0.001, max_rate=20, allow_interupt=True):    
    start_wait = time.time()

    # set event info to be empty
    # as this is dict we can change entries
    # directly without using
    # the global keyword
    event_info['type'] = ""
    event_info['description'] = ""
    event_info['time'] = -1

    n_proc = int(max_rate*interval)+1
    
    with ui_events() as ui_poll:
        keep_looping = True
        while keep_looping==True:
            # process UI events
            ui_poll(n_proc)

            # end loop if we have waited more than the timeout period
            if (timeout != -1) and (time.time() > start_wait + timeout):
                keep_looping = False
                
            # end loop if event has occured
            if allow_interupt==True and event_info['description']!="":
                keep_looping = False
                
            # add pause before looping
            # to check events again
            time.sleep(interval)
    
    # return event description after wait ends
    # will be set to empty string '' if no event occured
    return event_info

# this function lets buttons 
# register events when clicked
def register_event(btn):

    btn.disabled = True
    
    # display button description in output area
    event_info['type'] = "click"
    event_info['description'] = btn.description
    event_info['time'] = time.time()
    return

def register_text_input_event(text_input):
    event_info['type'] = "text_entry"
    event_info['description'] = text_input.value
    event_info['time'] = time.time()
    return

#Set up input box
def text_input(prompt=None):
    text_input = widgets.Text(description=prompt, style= {'description_width': 'initial'})
    import warnings
    warnings.filterwarnings("ignore", category=DeprecationWarning)
    text_input.on_submit(register_text_input_event)
    display(text_input)
    event = wait_for_event()
    text_input.disabled = True
    return event['description']

In [ ]:
#Download uploaded data
excel_url = 'https://docs.google.com/spreadsheets/d/1eLCnXnqxTn0HwMfFf3jnjmjoXBoXpVtbPEFGq-iKXqA/export?format=csv'
df = pd.read_csv(excel_url)
df['Correct'] = df['Correct rate'].str.rstrip('%').astype(float) / 100.0
total_rows = df.shape[0]

output = widgets.Output()

# Initialize global variables for game state
points = 0
current_level = 1
game_over = False
game_start = False

# Define cube images
cube_images = {
    1: Image("Cube1.png", width=600,height=400),
    2: Image("Cube2.png", width=600,height=400),
    3: Image("Cube3.png", width=600,height=400),
    4: Image("Cube4.png", width=600,height=400),
    5: Image("Cube5.png", width=600,height=400),
    6: Image("Cube6.png", width=600,height=400),
    7: Image("Cube7.png", width=600,height=400),
    8: Image("Cube8.png", width=600,height=400),
    9: Image("Cube9.png", width=600,height=400),
    10: Image("Cube10.png", width=600,height=400),
    11: Image("Cube11.png", width=600,height=400),
    12: Image("Cube12.png", width=600,height=400),
}

#Define correct answers
correct_answers = {
    1: 'd',
    2: 'b',
    3: 'c',
    4: 'b',
    5: 'd',
    6: 'd',
    7: 'b',
    8: 'a',
    9: 'b',
    10: 'a',
    11: 'd',
    12: 'a',
}

#Ask player to make up and ID
print("Enter your anonymised ID")
print("To generate an anonymous 4-letter unique user identifier please enter:")
print("- two letters based on the initials (first and last name) of a childhood friend")
print("- two letters based on the initials (first and last name) of a favourite actor / actress")
print("")
print("e.g. if your friend was called Charlie Brown and film star was Tom Cruise")
print("then your unique identifier would be CBTC")
name = input()
while len(name.strip()) != 4 or not name.isalpha():
    if len(name.strip()) != 4:
        print("Invalid input! User ID must be exactly 4 letters long.")
    elif not name.isalpha():
        print("Invalid input! User ID must consist only of letters.")
    name = input("Please enter a 4-letter User ID: ")

print(f"You entered:{name}")
time.sleep(1)
clear_output(wait=False)

#Ask player to input age, answer cannot be empty or below zero
age = input("Please enter your age: ")
while not age.isdigit() or int(age) <= 0:
    print("Invalid input! Age must be a positive number.")
    age = input("Please enter your age again: ")

print(f"You entered:{age}")
time.sleep(1)
clear_output(wait=False)

#Collect User Info
print("What is your ethnicity?")

# Create buttons with options
btna1 = widgets.Button(description='White')
btna2 = widgets.Button(description='Black or African American')
btna3 = widgets.Button(description='Asian')
btna4 = widgets.Button(description='Other')
btna5 = widgets.Button(description='Rather not say')

# Assign event handlers to buttons
btna1.on_click(register_event) 
btna2.on_click(register_event) 
btna3.on_click(register_event) 
btna4.on_click(register_event) 
btna5.on_click(register_event) 

display(btna1, btna2, btna3, btna4, btna5)
event_infoa = wait_for_event(timeout=60)

ethnicity = event_infoa['description']
print(f"User clicked: {event_infoa['description']}")

time.sleep(1)
clear_output(wait=True)

#Collect User Info
print("Please enter your biological sex:")

btnb1 = widgets.Button(description='Male')
btnb2 = widgets.Button(description='Female')
btnb3 = widgets.Button(description='Rather not say')

# Assign event handlers to buttons
btnb1.on_click(register_event) 
btnb2.on_click(register_event) 
btnb3.on_click(register_event) 

display(btnb1, btnb2, btnb3)
event_infob = wait_for_event(timeout=60)

sex = event_infob['description']
print(f"User clicked: {event_infob['description']}")

time.sleep(1)
clear_output(wait=True)

#Collect User Info
print("Did you have breakfast today:")
btnc1 = widgets.Button(description='Yes')
btnc2 = widgets.Button(description='No')
btnc3 = widgets.Button(description='Rather not say')

# Assign event handlers to buttons
btnc1.on_click(register_event) 
btnc2.on_click(register_event) 
btnc3.on_click(register_event) 

display(btnc1, btnc2, btnc3)
event_infoc = wait_for_event(timeout=60)

breakfast = event_infoc['description']
print(f"User clicked: {event_infoc['description']}")

time.sleep(1)
clear_output(wait=True)

#Collect User Info
print("How long did you sleep last night?")
btnd1 = widgets.Button(description='less than 5 hours')
btnd2 = widgets.Button(description='5-9 hours')
btnd3 = widgets.Button(description='more than 9 hours')
btnd4 = widgets.Button(description='Rather not say')

# Assign event handlers to buttons
btnd1.on_click(register_event) 
btnd2.on_click(register_event) 
btnd3.on_click(register_event) 
btnd4.on_click(register_event) 

display(btnd1, btnd2, btnd3, btnd4)
event_infod = wait_for_event(timeout=60)

sleep = event_infod['description']
print(f"User clicked: {event_infod['description']}")

time.sleep(1)
clear_output(wait=True)

#Set up data structure for storing player information
data = {
    'Name': name,
    'Age': age,
    'Ethnicity': ethnicity,
    'Sex': sex,
    'Breakfast': breakfast,
    'Sleep': sleep
}
results_json = dict()

# Introductory text widgets using widgets.HTML
Intro = widgets.HTML("<h1>Welcome to the Spatial Reasoning Test</h1>")
Intro2 = widgets.HTML("<h2>Identify a single 2D projection which cannot be made by rotating the 3D arrangement.</h2>")
Intro3 = widgets.HTML("<h2>You have 3 minutes to complete this 12-Level Test. ENJOY!</h2>")

# Group and display introductory elements
intro_box = widgets.VBox([Intro, Intro2, Intro3])
with output:
    display(intro_box)

#Check game duration, three minute limit
def check_time_limit():
    global game_over
    if time.time() - game_start_time > 180:  # 180 seconds = 3 minutes
        game_over = True
        end_game()
    
#Updates the dispay in the game 
#Shows current point, level, image, progress bar
def update_display(level):
    global start_time
    with output:
        clear_output(wait=True)
        progress_bar = widgets.IntProgress(value=level,min=0,max=12,description='',bar_style='info',style={'bar_color': 'blue'})
        display(widgets.HBox([widgets.HTML("<b>Progress:</b>"), progress_bar,widgets.HTML(f"<b>{level}/12</b>")]))
        display(widgets.HTML(f"<b>Points:</b> {points}"))
        if level in cube_images:
            display(cube_images[level])
        display(widgets.HBox(answer_buttons))
    start_time = time.time()

#Updates scores, tracks user response time, provide feedback
#Advance game to next question or end game if there are no more questions
def on_answer(option, level):
    global current_level, points, start_time, data_dic
    time_used = time.time() - start_time
    correct = option == correct_answers[level]
    data[f'Is question {level} correct'] = "Yes" if correct else "No"
    results_json[f'Is question {level} correct'] = "Yes" if correct else "No"
    results_json[f'Question {level} time'] = f"{time_used:.2f}s"
    points += 1 if correct else 0
    feedback = "Correct!" if correct else "Incorrect!"
    feedback += f" Point +{1 if correct else 0} | Time used: {time_used:.2f}s"
    with output:
        clear_output(wait=True)
        print(feedback)
        time.sleep(2)
    current_level += 1
    if current_level <= len(correct_answers):
        update_display(current_level)
    else:
        end_game()
0
#When button is clicked, it checks whether the game is still active
def on_button_click(b):
    if not game_over:
        check_time_limit()  # Check if time limit reached before processing the answer
        if not game_over:
            on_answer(b.description.lower(), current_level)

# Initialize and create answer buttons
#When clicked, buttons trigger the 'on_button_click' function
answer_buttons = [widgets.Button(description=answer) for answer in ['A', 'B', 'C', 'D']]
for button in answer_buttons:
    button.on_click(on_button_click)

#Start button allows players to start the game when they are ready
start_button = widgets.Button(description='Start Test')

#'start_test' function initializes and restarts to its initial conditions
#'Global' resets variables such as 'current_level', 'points', 'game_over', 'game_start_time'
def start_test(b):
    global current_level, points, game_over, game_start_time, timer_thread, game_start, timer
    with output:
        clear_output(wait=False)
    game_start_time = time.time()  # Reset game start time each time the test starts
    current_level = 1
    points = 0
    game_over = False
    check_time_limit()  # Initial check to see if we're already past the time limit
    if not game_over:
        game_start = True
        update_display(current_level)

#Connects 'start_test' function to 'start_button's click event
start_button.on_click(start_test)
with output:
    display(start_button)
display(output)  # Ensure output is displayed for feedback

#Upon end of the game    
def end_game():
    global game_over, data
    game_over = True  #Indicate the game has ended
    print("end_game")
    for level in range(current_level, 13): #Fills in default values for unanswered questions
        data[f'Is question {level} correct'] = "No" #Marks unanswered questions as incorrect
        results_json[f'Is question {level} correct'] = "No" #Marks unanswered questions as incorrect
        results_json[f'Question {level} time'] = "0.00s" #Sets unanswered questions with time spent of 0.00seconds
        
    percentage_score = (points / len(correct_answers)) * 100 #Calculate percentage score
    data[f'Correct rate'] = f'{percentage_score:.2f}%' #Upload percentage score to 'data' which is stored in the Google Form
    results_json[f'Correct rate'] = f'{percentage_score:.2f}%' #Upload percentage score to 'results_json' in the google form
    data["results_json"] =json.dumps(results_json)
    correct = round(percentage_score / 100, 2) #Round percentage score to 2dp.
    smaller_count = (df['Correct'] < correct).sum() #Compares player's score against database of 40 players
    smaller_ratio = smaller_count / total_rows * 100
    with output: #Presents player with summary of their performance:total points,percentage score,and how they did compared to others
        clear_output(wait=True)
        display(widgets.HTML(f"<h3>Times up Thank You for Participating! You scored {points} points out of {len(correct_answers)}.</h3>"))
        display(widgets.HTML(f"<h3>This gives you a score of {percentage_score:.2f}%, you did better than {smaller_ratio:.2f}% of the players.</h3>"))
        SRT_df = pd.read_csv("Spatial Reasoning Data.csv", index_col='Name')
        results = SRT_df['Total score']
    
        mean_result = np.mean(results)
        median_result = np.median(results)
        std_dev = np.std(results)

    # Plot histogram
        plt.hist(results, bins=10, edgecolor='black')
        plt.axvline(x=mean_result, color='red', linestyle='--', label=f'Mean: {mean_result:.2f}')
        plt.axvline(x=median_result, color='green', linestyle='--', label=f'Median: {median_result:.2f}')
        plt.xlabel('Test Results')
        plt.ylabel('Frequency')
        plt.title('Distribution of Test Results')
        plt.legend()
    
    # Add annotation for user's result
        plt.annotate('Your Result', xy=(percentage_score, 1), xytext=(percentage_score + 5, 3),
                 arrowprops=dict(facecolor='black', shrink=0.05))
    
    # Show plot
        plt.show()
    
        print("\nPlease read:") #Asking for consent for data use
        print("")
        print("We wish to record your response data to an anonymised public data repository. ")
        print("Your data will be used for educational teaching purposes practising data analysis and visualisation.")
        print("")
        print("Please type   yes   in the box below if you consent to the upload.")
        result = text_input("> ")
        if result.lower() == "yes": #If player consents data use, data is uploaded to Google form
            print("Thanks for your participation.")
            print("Please contact a.fedorec@ucl.ac.uk")
            print("If you have any questions or concerns")
            print("regarding the stored results.")
            form_url = 'https://docs.google.com/forms/d/e/1FAIpQLSc09g4wn9F-PQvVf4nKFeSIXgAiZwbFqKEps_2PzByODtAKig/viewform'
            send_to_google_form(data,form_url)
        else:
            # end code execution by raising an exception
            raise(Exception("User did not consent to continue test."))

        clear_output(wait=True)